<a href="https://colab.research.google.com/github/abxda/python/blob/main/UP_Semana_4_Viernes_y_S%C3%A1bado_02_Geomarketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmnx descartes

In [ ]:
!pip install alphashape

In [ ]:

#Cargar Bibliotecas
import folium
from shapely.geometry import Point, LineString, Polygon
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
from descartes import PolygonPatch
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
import requests
from zipfile import ZipFile
import time
import os
import shutil
ox.config(log_console=True, use_cache=True)
from IPython.display import IFrame
from alphashape import alphashape


In [ ]:
#Funciones de apoyo
def download(url,dir):
    time.sleep(5)
    chunk_size = 1024
    r = requests.get(url, stream = True)
    total_size = int(r.headers['content-length'])
    filename = url.split('/')[-1]
    with open(dir+filename, 'wb') as f:
        for data in tqdm(iterable = r.iter_content(chunk_size = chunk_size), total = total_size/chunk_size, unit = 'KB'):
            f.write(data)

def extract_shapefile(states,directory,shp_dir,shape_type):
    for i in range(32):
        try:
            estado = states[i]
            file = str(i+1).zfill(2)
            zip_file = directory+estado
            shp_file = f'conjunto_de_datos/{file}{shape_type}.shp'
            cpg_file = f'conjunto_de_datos/{file}{shape_type}.cpg'
            dbf_file = f'conjunto_de_datos/{file}{shape_type}.dbf'
            prj_file = f'conjunto_de_datos/{file}{shape_type}.prj'
            shx_file = f'conjunto_de_datos/{file}{shape_type}.shx'
            with ZipFile(zip_file, 'r') as zip:
                zip.extract(shp_file,shp_dir)
                zip.extract(dbf_file,shp_dir)
                zip.extract(prj_file,shp_dir)
                zip.extract(shx_file,shp_dir)
                try:
                    zip.extract(cpg_file,shp_dir)
                except:
                    with open(shp_dir+cpg_file, 'w') as out_file:
                        out_file.write("ISO 88591")
        except:
            pass

def extract_shapefile_denue(states,directory,shp_dir,shape_type):
    for i in range(32):
        try:
            estado = states[i]
            file = str(i+1).zfill(2)
            zip_file = directory+f'denue_{file}_shp.zip'
            shp_file = f'conjunto_de_datos/denue_inegi_{file}{shape_type}.shp'
            cpg_file = f'conjunto_de_datos/denue_inegi_{file}{shape_type}.cpg'
            dbf_file = f'conjunto_de_datos/denue_inegi_{file}{shape_type}.dbf'
            prj_file = f'conjunto_de_datos/denue_inegi_{file}{shape_type}.prj'
            shx_file = f'conjunto_de_datos/denue_inegi_{file}{shape_type}.shx'
            with ZipFile(zip_file, 'r') as zip:
                zip.extract(shp_file,shp_dir)
                zip.extract(dbf_file,shp_dir)
                zip.extract(prj_file,shp_dir)
                zip.extract(shx_file,shp_dir)
                try:
                    zip.extract(cpg_file,shp_dir)
                except:
                    with open(shp_dir+cpg_file, 'w') as out_file:
                        out_file.write("ISO 88591")
        except:
            pass

In [ ]:

# Estructura de Directorios
os.makedirs("./inegi/ccpvagebmza/csv/conjunto_de_datos")
os.makedirs("./inegi/mgccpv/shp/m/conjunto_de_datos")
os.makedirs("./inegi/mgccpv/gpkg/")
os.makedirs("./inegi/denue/conjunto_de_datos/")

In [ ]:
estados_geo = ["01_aguascalientes.zip","02_bajacalifornia.zip","03_bajacaliforniasur.zip","04_campeche.zip","05_coahuiladezaragoza.zip","06_colima.zip","07_chiapas.zip","08_chihuahua.zip","09_ciudaddemexico.zip","10_durango.zip","11_guanajuato.zip","12_guerrero.zip","13_hidalgo.zip","14_jalisco.zip","15_mexico.zip","16_michoacandeocampo.zip","17_morelos.zip","18_nayarit.zip","19_nuevoleon.zip","20_oaxaca.zip","21_puebla.zip","22_queretaro.zip","23_quintanaroo.zip","24_sanluispotosi.zip","25_sinaloa.zip","26_sonora.zip","27_tabasco.zip","28_tamaulipas.zip","29_tlaxcala.zip","30_veracruzignaciodelallave.zip","31_yucatan.zip","32_zacatecas.zip"]
estados_num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13 ,14 ,15, 16, 17, 18, 19, 20, 21, 22, 23 ,24, 25, 26, 27, 28, 29, 30, 31, 32]
estados_denue = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13" ,"14" ,"15_1","15_2", "16", "17", "18", "19", "20", "21", "22", "23" ,"24", "25", "26", "27", "28", "29", "30", "31", "32"]

In [ ]:
# Descarga los Datos del Censo
directory= "./inegi/ccpvagebmza/"
estado = "01"
ageb_mza = f'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/ageb_mza_urbana_{estado}_cpv2020_csv.zip'
download(ageb_mza,directory)

# Descarga los datos del Margo Geoestadístico
directory= "./inegi/mgccpv/"
url_mgccpv = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/"
state = "01_aguascalientes.zip"
state_file = url_mgccpv+state
download(state_file,directory)

# Descarga los datos del DENUE
directory= "./inegi/denue/"
estado = "01"
url_denue = f'https://www.inegi.org.mx/contenidos/masiva/denue/denue_{estado}_shp.zip'
download(url_denue,directory)

In [ ]:
# Extracción de Datos
states = ["01_aguascalientes.zip"]#,"02_bajacalifornia.zip","03_bajacaliforniasur.zip","04_campeche.zip","05_coahuiladezaragoza.zip","06_colima.zip","07_chiapas.zip","08_chihuahua.zip","09_ciudaddemexico.zip","10_durango.zip","11_guanajuato.zip","12_guerrero.zip","13_hidalgo.zip","14_jalisco.zip","15_mexico.zip","16_michoacandeocampo.zip","17_morelos.zip","18_nayarit.zip","19_nuevoleon.zip","20_oaxaca.zip","21_puebla.zip","22_queretaro.zip","23_quintanaroo.zip","24_sanluispotosi.zip","25_sinaloa.zip","26_sonora.zip","27_tabasco.zip","28_tamaulipas.zip","29_tlaxcala.zip","30_veracruzignaciodelallave.zip","31_yucatan.zip","32_zacatecas.zip"]
# Variables e indicadores del Censo de Población 2020
# CSV Files
directory= "./inegi/ccpvagebmza/"
csv_dir = directory+"csv/conjunto_de_datos/"
estado = "01"
zip_file = directory+f'ageb_mza_urbana_{estado}_cpv2020_csv.zip'
csv_file = f'ageb_mza_urbana_{estado}_cpv2020/conjunto_de_datos/conjunto_de_datos_ageb_urbana_{estado}_cpv2020.csv'
with ZipFile(zip_file, 'r') as zip:
    zipInfo = zip.getinfo(csv_file)
    zipInfo.filename = os.path.basename(csv_file)
    zip.extract(zipInfo, csv_dir)

# Localidades
shape_type = "l"
directory= "./inegi/mgccpv/"
shp_dir = directory+"shp/m/"
extract_shapefile(states,directory,shp_dir,shape_type)

# Manzanas
shape_type = "m"
directory= "./inegi/mgccpv/"
shp_dir = directory+"shp/m/"
extract_shapefile(states,directory,shp_dir,shape_type)

# Caserío disperso
shape_type = "cd"
directory= "./inegi/mgccpv/"
shp_dir = directory+"shp/m/"
extract_shapefile(states,directory,shp_dir,shape_type)

# Polígono externo de manzana
shape_type = "pem"
directory= "./inegi/mgccpv/"
shp_dir = directory+"shp/m/"
extract_shapefile(states,directory,shp_dir,shape_type)

# DENUE
shape_type = "_"
directory= "./inegi/denue/"
shp_dir = directory
extract_shapefile_denue(states,directory,shp_dir,shape_type)

In [ ]:
#Union de Datos de Aguascalientes
estado = '01'
print("procesando estado: "+str(estado))
locdf = gpd.read_file(f"./inegi/mgccpv/shp/m/conjunto_de_datos/{estado}l.shp").to_crs("EPSG:4326")
denuedf = gpd.read_file(f"./inegi/denue/conjunto_de_datos/denue_inegi_{estado}_.shp").to_crs("EPSG:4326")

gpdf = gpd.read_file(f"./inegi/mgccpv/shp/m/conjunto_de_datos/{estado}m.shp").to_crs("EPSG:4326")
cddf = gpd.read_file(f"./inegi/mgccpv/shp/m/conjunto_de_datos/{estado}cd.shp").to_crs("EPSG:4326")
pemdf = gpd.read_file(f"./inegi/mgccpv/shp/m/conjunto_de_datos/{estado}pem.shp").to_crs("EPSG:4326")
df = pd.read_csv(f"./inegi/ccpvagebmza/csv/conjunto_de_datos/conjunto_de_datos_ageb_urbana_{estado}_cpv2020.csv",na_values=['N/A','N/D','*'])
df['CVEGEO'] = df.apply(lambda row: str(row['ENTIDAD']).zfill(2) + str(row['MUN']).zfill(3)+ str(row['LOC']).zfill(4)+ str(row['AGEB']).zfill(4)+ str(row['MZA']).zfill(3), axis=1)
df_geo_censo = pd.merge(df, gpdf, how = 'left').drop(["CVE_ENT", "CVE_MUN", "CVE_LOC", "CVE_AGEB", "CVE_MZA"], axis = 1)
df_geo_censo = df_geo_censo.drop(df[df.MZA == 0].index)
df_geo_censo = gpd.GeoDataFrame(df_geo_censo, geometry="geometry")

df_full = df_geo_censo[df_geo_censo['geometry'].notnull()]
df1 = df_geo_censo[df_geo_censo['geometry'].isnull()]
df1 = df1.drop(["TIPOMZA","AMBITO","geometry"], axis = 1)
df_geo_dif = pd.merge(df1, cddf, how = 'left').drop(["CVE_ENT", "CVE_MUN", "CVE_LOC", "CVE_AGEB", "CVE_MZA"], axis = 1)
pemdf = pemdf.rename(columns={'geometry': 'geometry_pem'})
pemdf = pemdf.drop(["CVE_ENT", "CVE_MUN", "CVE_LOC", "CVE_AGEB", "CVE_MZA"], axis = 1)
df_geo_dif = pd.merge(df_geo_dif, pemdf, how = 'left')
df_geo_dif['geometry'] = df_geo_dif.apply(lambda row : row['geometry_pem'] if row['geometry_pem'] != None else row['geometry'], axis = 1)
df_geo_dif =  df_geo_dif.drop(["geometry_pem"], axis = 1)
final_shape = pd.concat([df_full,df_geo_dif],ignore_index=True)
final_shape["AGEB"] = final_shape.AGEB.apply(str)
print("guardando datos del estado: "+str(estado))
final_shape.to_file(f"./inegi/mgccpv/gpkg/cpv2020_{estado}.gpkg", layer='cpv2020', driver="GPKG")
print("guardando datos del poligono de localidad del estado: "+str(estado))
locdf.to_file(f"./inegi/mgccpv/gpkg/cpv2020_{estado}.gpkg", layer='localidades2020', driver="GPKG")
print("guardando datos del DENUE del estado: "+str(estado))
denuedf.to_file(f"./inegi/mgccpv/gpkg/cpv2020_{estado}.gpkg", layer='denue2020', driver="GPKG")

In [ ]:
localidades = gpd.read_file(f"./inegi/mgccpv/gpkg/cpv2020_{estado}.gpkg", layer='localidades2020', driver="GPKG")
ags = localidades.loc[localidades['CVEGEO'] == '010010001']
ags['geometry'].array[0]

In [ ]:
# Modo de transporte
mode = "walk"
# Crear el grafo que cubre el polígono segun el modo de transporte
G = ox.graph_from_polygon(ags['geometry'].array[0], simplify=True, network_type=mode)
# Dibujamos el resultado para tener una idea de lo que se descargó
fig, ax = ox.plot_graph(G, save=True);
plt.tight_layout()
plt.show()

In [ ]:
denue = gpd.read_file(f"./inegi/mgccpv/gpkg/cpv2020_{estado}.gpkg", layer='denue2020', driver="GPKG")
#Usamos los códigos de 464111 para Farmacias sin minisúper , 464112 para Farmacias con minisúper
farmacias = denue.loc[denue['codigo_act'].isin(['464111','464112'])]
farmacias

In [ ]:
isochrone_polys = []

G_UTM = ox.project_graph(G)
trip_time = 10
travel_speed = 4
meters_per_minute = travel_speed * 1000 / 60

for u, v, k, data in G.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

for indice in farmacias.index:
    point = farmacias['geometry'][indice]
    p = point.y, point.x
    point_of_interest = point_of_interest = ox.distance.nearest_nodes(G, point.x, point.y,return_dist=False)
    subgraph = nx.ego_graph(G_UTM, point_of_interest, radius=trip_time, distance='time')
    #node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    node_points = [(data['x'], data['y']) for node, data in subgraph.nodes(data=True)]


    alpha_value = 0.008  # Ajusta este valor para controlar la concavidad
    bounding_poly = alphashape(node_points, alpha=alpha_value)
    #bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isochrone_polys.append(bounding_poly)
gdf = gpd.GeoDataFrame({"id": farmacias['id']}, geometry= isochrone_polys)
gdf_edges = ox.graph_to_gdfs(G_UTM, nodes=False, fill_edge_geometry=True)
gdf.crs = gdf_edges.crs
gdf.to_crs("EPSG:4326").to_file(f"./inegi/mgccpv/gpkg/isocronas_{estado}.gpkg", layer='isocronas', driver="GPKG")


In [ ]:
alpha_value = 0.008  # Ajusta este valor para controlar la concavidad
bounding_poly = alphashape(node_points, alpha=alpha_value)
bounding_poly

In [ ]:
alpha_value = 0.0008  # Ajusta este valor para controlar la concavidad
bounding_poly = alphashape(node_points, alpha=alpha_value)
bounding_poly

In [ ]:
manzanas = gpd.read_file(f"./inegi/mgccpv/gpkg/cpv2020_{estado}.gpkg", layer='cpv2020', driver="GPKG")
manzanas.crs = "EPSG:4326"
manzanas_ags = manzanas.loc[manzanas['NOM_LOC'].isin(['Aguascalientes'])]
isocronas = gpd.read_file(f"./inegi/mgccpv/gpkg/isocronas_{estado}.gpkg", layer='isocronas', driver="GPKG")
isocronas["Negocio"] = "Farmacia"
dfsjoin = gpd.sjoin(isocronas, manzanas_ags, how="left", predicate='contains')
dfpivot = pd.pivot_table(dfsjoin,index='CVEGEO',columns='Negocio',aggfunc={'Negocio':len})
dfpivot.columns = dfpivot.columns.droplevel()
manzanas_ags_iso = manzanas_ags.merge(dfpivot, how='left', on='CVEGEO')
manzanas_ags_iso.fillna(value=0, inplace=True)
manzanas_ags_iso.to_file(f"./inegi/mgccpv/gpkg/manzanas_iso_{estado}.gpkg", layer='mza_iso', driver="GPKG")